In [1]:
from src.flows.news_processing.pipelines.news_pipeline import process_news

In [2]:
process_news()

16:44:51.435 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8578
See https://docs.prefect.io/3.0/manage/self-host#self-host-a-prefect-server for more information on running a dedicated Prefect server.

16:44:56.870 | INFO    | Flow run 'stirring-angelfish' - Beginning flow run 'stirring-angelfish' for flow 'crypto_news_processing'

16:44:57.194 | INFO    | Task run 'import_news-fe2' - Calling NewsImportEndpoint...

2025-01-19 16:44:58,464 - ImportNewsEndpoint - INFO - Fetching News...
2025-01-19 16:44:58,466 - GetCryptoNews - INFO - Fetching articles between 2025-01-18 and 2025-01-19


Fetching news:   0%|          | 0/2 [00:00<?, ?it/s]

...

2025-01-19 16:45:07,387 - ArticleCleanEndpoint - INFO - Cleaning Data...
2025-01-19 16:45:07,465 - ArticleCleanEndpoint - INFO - Feature Engineering Data...
2025-01-19 16:45:07,943 - DeltaLakeManager - INFO - Table: cleaned_data - Persisting data...
2025-01-19 16:45:08,423 - DeltaLakeManager - INFO - Table: cleaned_data - Merged data: 2 rows inserted, 0 rows updated
2025-01-19 16:45:08,423 - DeltaLakeManager - INFO - Table: article_status_data - Persisting data...
2025-01-19 16:45:08,459 - DeltaLakeManager - INFO - Table: article_status_data - Merged data: 0 rows inserted, 2 rows updated
2025-01-19 16:45:08,459 - ArticleCleanEndpoint - INFO - Successfully persisted 2 cleaned articles


16:45:08.459 | INFO    | Task run 'clean_articles-44d' - Cleaning completed: {'status': 'success', 'articles_cleaned': 2}

16:45:08.466 | INFO    | Task run 'clean_articles-44d' - Finished in state Completed()

16:45:08.509 | INFO    | Flow run 'stirring-angelfish' - Finished in state Completed()

{'import': {'status': 'success',
  'new_articles': 2,
  'date_range': '2025-01-18 to 2025-01-19'},
 'scrape': {'status': 'success',
  'articles_scraped': 2,
  'successful_scrapes': 2,
  'success_rate': '100.0%'},
 'clean': {'status': 'success', 'articles_cleaned': 2}}

In [ ]:
from src.collect.news_fetcher import *

In [ ]:
cls = NewsImportEndpoint()

In [ ]:
last_fetch_date = cls._get_last_fetch_date()

In [ ]:
start_date = (last_fetch_date - timedelta(days=1))

In [ ]:
end_date = pd.Timestamp.now(tz = 'US/Eastern')

In [ ]:
news_metadata = cls.fetcher.fetch_news(start_date, end_date)

In [ ]:
existing_articles = cls.deltalake.read_table(
    table_name=TableNames.METADATA_ARTICLES.value,
    columns=['news_id']
)

In [ ]:
news_metadata = news_metadata[
    ~news_metadata['news_id'].isin(existing_articles['news_id'])
]

In [ ]:
news_metadata

In [ ]:
cls.deltalake.write_table(
    table_name=TableNames.METADATA_ARTICLES.value,
    df=news_metadata
)

In [ ]:
news_metadata2 = cls.deltalake.read_table(
    table_name=TableNames.METADATA_ARTICLES.value,
)

In [ ]:
print("Tickers column type:", type(news_metadata['tickers'].iloc[0]))
print("Topics column type:", type(news_metadata['topics'].iloc[0]))

In [ ]:
news_metadata2['topics'].iloc[0]